In [25]:

mainUrl = "http://www.sec.gov/Archives/edgar/data/"
CIK = input("Enter CIK (eg :0000051143)")
 # DAN has to come from some search results as per problem case
DAN = input("Enter document accession number -DAN (eg :000005114313000007 )")
CIK = CIK.strip().strip("0")
DAN = DAN.strip()
partCIK = DAN[0:10]+"-"
partDAN = DAN[10:12]+"-"
lastPart = DAN[12:18]+"-index.html"

completeURL = mainUrl+CIK+"/"+DAN+"/"+partCIK+partDAN+lastPart
print(completeURL)


Enter CIK (eg :0000051143)0000051143
Enter document accession number -DAN (eg :000005114313000007 )000005114313000007 
http://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.html


In [37]:
from bs4 import BeautifulSoup
from collections import namedtuple
import urllib.request as ur
from collections import namedtuple
import pprint
import csv
import re

class Page:

    def __init__(self, url):
        """
        Retrieves and stores the urllib.urlopen object for a given url
        """

        self.link = ur.urlopen(url)

    def get_tables(self):
        """
        Extracts each table on the page and places it in a dictionary.
        Converts each dictionary to a Table object. Returns a list of
        pointers to the respective Table object(s).
        """

        raw_html = self.link.read()
        soup = BeautifulSoup(raw_html, "html.parser")
        tables = soup.findAll("table")

        # have to extract each entry using nested loops
        table_list = []
        for table in tables:
            # empty dictionary each time represents our table
            table_dict = {}
            rows = table.findAll("tr")
            # count will be the key for each list of values
            count = 0
            for row in rows:
                value_list = []
                entries = row.findAll("td")
                for entry in entries:
                    # fix the encoding issues with utf-8
                    entry = entry.text.encode("utf-8", "ignore")
                    #strip_unicode = re.compile("([^-_a-zA-Z0-9!@#%&=,/'\";:~`\$\^\*\(\)\+\[\]\.\{\}\|\?\<\>\\]+|[^\s]+)")
                    #entry = strip_unicode.sub(" ", entry.decode(encoding="utf-8"))
                    #print(len(entry))
                    value_list.append(entry)
                # we don't want empty data packages
                print(value_list[0])
                if len(value_list) > 0:
                    table_dict[count] = value_list
                    count += 1

            table_obj = Table(table_dict)
            #table_obj.show_table()
            table_list.append(table_obj)

        return table_list

    def save_tables(self, tables, ignore_small=False):
        """
        Takes an input a list of table objects and saves each
        table to csv format. If ignore_small is True,
        we ignore any tables with 5 entries or fewer. 
        """

        counter = 1
        for table in tables:
            if ignore_small:
                if table.get_metadata().num_entries > 5:
                    name = "table" + str(counter)
                    table.save_table(name)
                    counter += 1
            else:
                name = "table" + str(counter)
                table.save_table(name)
                counter += 1
                
                
Metadata = namedtuple("Metadata", "num_cols num_entries")

class Table:

    def __init__(self, data):
        """
        Stores a given table as a dictionary. The keys are the headings and the
        values are the data, represented as lists.
        """
        self.table_data = data

    def get_metadata(self):
        """
        Returns a Metadata object that contains the number of columns
        and the total number of entries.
        """

        col_headings = self.table_data.keys()
        num_cols = len(col_headings)
        num_entries = 0

        for heading in col_headings:
            num_entries += len(self.table_data[heading])

        return Metadata(
            num_cols = num_cols,
            num_entries = num_entries
        )

    def show_table(self):
        """
        Prints a formatted table to the command line using pprint
        """
        pprint.pprint(self.table_data, width=1)

    def save_table(self, name):
        """
        Saves a table to csv format under the given file name. 
        File name should omit the extension.
        """
        fname = name + ".csv"

        with open(fname, 'w',encoding='utf8',newline='') as outf:
            w = csv.writer(outf, dialect="excel")
            li = self.table_data.values()
            print(len(li))
            w.writerows(li)
# url that contains the tables we want
url="https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm"

# convert to a page object
#page = Page(url)
# get the tables
#tables = page.get_tables()

# save the tables
#page.save_tables(tables)

# you can change the name that it saves the table to by calling save_table on the table object itself:
# don't include the extension in the file name
#tables[0].save_table("customName")
#url = "https://www.eia.gov/electricity/monthly/epm_table_grapher.cfm?t=epmt_1_1"
page = Page(url)
tables = page.get_tables()
page.save_tables(tables, ignore_small=False)

b'\nNew York\n'
b'\n(State of incorporation)\n'
b'\nArmonk, New York\n'
b'\n(Address of principal executive offices)\n'
b'\nLarge accelerated filer \xc3\xbd\xc2\xa0\n'
b'\n\xc2\xa0\n'
b'\nNon-accelerated filer\xc2\xa0 \xc2\xa8\xc2\xa0\n'
b'\n(Do not\n  check if a smaller reporting company)\n'
b'\n\xc2\xa0\n'
b'\nPart I - Financial Information:\n'
b'\n\xc2\xa0\n'
b'\nItem\n  1. Consolidated Financial Statements:\n'
b'\n\xc2\xa0\n'
b'\n\xc2\xa0\n'
b'\nConsolidated Statement of Earnings for the three and nine months ended\n  September 30, 2013 and 2012\n'
b'\n\xc2\xa0\n'
b'\nConsolidated Statement of Comprehensive Income for the three and nine\n  months ended September 30, 2013 and 2012\n'
b'\n\xc2\xa0\n'
b'\nConsolidated\n  Statement of Financial Position at September 30, 2013 and December 31, 2012 \xc2\xa0\n'
b'\n\xc2\xa0\n'
b'\nConsolidated\n  Statement of Cash Flows for the nine months ended September 30, 2013 and 2012\n'
b'\n\xc2\xa0\n'
b'\nConsolidated\n  Statement of Changes in Equ

PermissionError: [Errno 13] Permission denied: 'table111.csv'